<font color="#CA0032"><h1 align="left">**Redes Generativas Adversariales (GANs)**</h1></font>

<font color="#6E6E6E"><h1 align="left">**Creación de imágenes nuevas con GANs no profundas**</h1></font>

<h2 align="left">Manuel Sánchez-Montañés</h2>

<font color="#6E6E6E"><h2 align="left">manuel.smontanes@gmail.com</h2></font>

In [ ]:
COLAB                  = True
SAVE_INTERMEDIATE_DATA = False

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import LeakyReLU, BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

%matplotlib inline

In [ ]:
np.random.seed(1000)

# Tamaño del espacio latente
randomDim = 20

# Carga de datos
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float32) / 255 # para que esté entre 0 y 1
X_train = 2*X_train - 1 # para que esté entre -1 y 1
X_train = X_train.reshape(-1, 28*28)

X_train = X_train[y_train>=5] # para simplificar se entrena sólo con los dígitos 5,6,7,8,9
y_train = y_train[y_train>=5] # ídem

In [ ]:
X_train.shape

In [ ]:
X_train.min(), X_train.max()

In [ ]:
ind = 100
plt.figure(figsize=(3,3))
plt.imshow(X_train[ind].reshape(28,28), cmap="gray")
plt.title("Clase {}".format(y_train[ind]));

In [ ]:
ind = 202
plt.figure(figsize=(3,3))
plt.imshow(X_train[ind].reshape(28,28), cmap="gray")
plt.title("Clase {}".format(y_train[ind]));

In [ ]:
ind = 20000
plt.figure(figsize=(3,3))
plt.imshow(X_train[ind].reshape(28,28), cmap="gray")
plt.title("Clase {}".format(y_train[ind]));

In [ ]:
X_train.shape

**Optimizadores**

In [ ]:
# optimizador para el generador:
adam_gen  = Adam(lr=0.0002, beta_1=0.5) # lr por defecto: 0.001

# optimizador para el discriminador:
adam_disc = Adam(lr=0.0002/2, beta_1=0.5)

In [ ]:
X_train.shape

**Red generadora ("generator")**

In [ ]:
randomDim # tamaño del espacio latente

Función de activación estándar en Deep Learning: ReLU

In [ ]:
def relu(z): # definición "a mano"
    return z*(z>0)

z = np.linspace(-10,10,100) # genero array de 100 puntos que recorren intervalo entre -10 y 10
plt.plot(z, relu(z))
plt.title("ReLU");

Función de activación en GANs: Leaky ReLU

In [ ]:
def leakyrelu(z):
    return z*(z>0) - 0.2*np.abs(z)*(z<0)

z = np.linspace(-10,10,100)
plt.plot(z, leakyrelu(z))
plt.title("Leaky ReLU");

In [ ]:
# entrada: randomDim dimensiones
# salida:  784 valores (entre -1 y 1)
#
# Función de activación estándar en DL:
# ReLU(x): max(0,x)
# En GANs:
# LeakyReLU(x,0.2):
#     si x>0: salida = x
#     si x<0: salida = 0.2*x = -0.2*abs(x)

generator = Sequential()
generator.add(Dense(64,input_shape=randomDim))    # input_dim = tamaño de la entrada / del espacio latente
generator.add(LeakyReLU(0.2))                     # función de activación
generator.add(Dense(784, activation='tanh'))      # tanh: tangente hiperbólica
generator.summary()

In [ ]:
type( (20) ), type( (20,))

**Red discriminadora ("discriminator")**

In [ ]:
X_train.shape

In [ ]:
discriminator = Sequential()

# Definir entrada de la red
discriminator.add(Dense(64, input_shape=(784,)))     # entrada: 784 valores (entre -1 y 1); 784 = 28*28
discriminator.add(LeakyReLU(0.2))                    # función de activación
discriminator.add(Dropout(0.3))                      # dropout: 30% de neuronas se desactivan aleatoriamente
'''# Definir capas ocultas (aqui quitado por tener un modelo más simple y rapido)
discriminator.add(Dense(16))                         # capa oculta
discriminator.add(LeakyReLU(0.2))                    # función de activación
discriminator.add(Dropout(0.3))                      # dropout: 30% de neuronas se desactivan aleatoriamente'''
# Definir salida de la red (si es falsa o verdadera = 1)
discriminator.add(Dense(1, activation='sigmoid'))    # salida: 1 valor (entre 0 y 1)
# Compilar la red
discriminator.compile(loss='binary_crossentropy',    # función de pérdida
                      optimizer=adam_disc)           # optimizador
discriminator.summary()

**Sistema combinado (GAN) generador+discriminador congelado**

In [ ]:
# completar
discriminator.trainable = False                 # congelo discriminador;  para que no se entrene el discriminador
ganInput = Input(shape=(randomDim,))            # entrada: espacio latente
ganOutput = discriminator(generator(ganInput))  # salida: discriminador de la salida del generador

gan = Model(inputs=ganInput, outputs=ganOutput) # modelo: entrada = espacio latente; salida = discriminador de la salida del generador
gan.compile(loss='binary_crossentropy',         # función de pérdida
            optimizer=adam_gen)                  # optimizador del generador porque es el que se entrena
gan.summary()

In [ ]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 3))
    plt.plot(range(1,len(dLosses)+1), dLosses,
             label='Discriminitive loss', linewidth=3)
    plt.plot(range(1,len(gLosses)+1), gLosses,
             label='Generative loss', linewidth=3)
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.legend(fontsize=14)
    if not COLAB:
        plt.savefig('./images/gan_loss_epoch_{}.png'.format(epoch))
    plt.show()

# Create a wall of generated images
def plotGeneratedImages(epoch, examples=100,
                        dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(len(generatedImages)):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    if SAVE_INTERMEDIATE_DATA:
        plt.savefig('./images/gan_generated_image_epoch_{}.png'.format(epoch))
    plt.show()


def plotImages(images, nrows, ncols, figsize):
    plt.figure(figsize=figsize)
    for i in range(images.shape[0]):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(images[i].reshape(28,28), interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

    
# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator.save('./models/gan_generator_epoch_{}.h5'.format(epoch))
    discriminator.save('./models/gan_discriminator_epoch_{}.h5'.format(epoch))

In [ ]:
if SAVE_INTERMEDIATE_DATA:
    os.makedirs("./images", exist_ok=True)
    os.makedirs("./models", exist_ok=True)

In [ ]:
len(X_train)

In [ ]:
dLosses = [] # histórico de los valores de la función de coste del discriminador
gLosses = [] # histórico de los valores de la función de coste del generador

In [ ]:
epochs = 200
batchSize=128

batchCount = len(X_train) // batchSize
batchCount

In [ ]:
batchSize*[0.9] + batchSize*[0.1]

In [ ]:
print('Epochs:', epochs)
print('Batch size:', batchSize)
print('Batches per epoch:', batchCount)

for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batchCount)):

        # ** EMPIEZA MINI-ENTRENAMIENTO DISCRIMINADOR: **
        
        # Genero entrada aleatoria al generador para batchSize (128) imágenes:
        noise = np.random.normal(0, 1, size=[batchSize, randomDim]) # distribución Gaussiana de media 0 y std 1
        
        # Genero imágenes falsas a través del generator:
        generatedImages = generator.predict(noise)

        # Selecciono al azar batchSize (128) imágenes reales
        imageBatch = X_train[np.random.randint(0, len(X_train), size=batchSize)]

        # Genero un X donde las 128 primeras imágenes son reales y las 128 siguientes fake
        X = np.concatenate([imageBatch, generatedImages])
        
        # Genero las etiquetas para estas 128+128 imágenes:
        # 128 "casi unos" (clase "real") seguidos de 128 "casi ceros" (clase "fake")
        yDis = np.array(batchSize*[0.9] + batchSize*[0.1])
        
        # Descongelo el discriminador:
        discriminator.trainable = True

        # Entreno discriminador
        dloss = discriminator.train_on_batch(X, yDis)
        
        # ** TERMINA MINI-ENTRENAMIENTO DISCRIMINADOR **

        
           
        # ** EMPIEZA MINI-ENTRENAMIENTO GENERADOR: **
        
        # Genero randomDim variables latentes (ruido) de entrada al generador
        # por cada una de las batchSize imágenes que quiero generar:
        noise = np.random.normal(0, 1, size=[batchSize, randomDim])

        # Genero etiquetas que deseo que el discriminador genere al pasarle
        # las imágenes creadas por el generador (deseo engañarle, con lo que
        # la salida deseada es 0.9, "casi real")
        yGen = np.array(batchSize*[0.9])

        # Congelo el discriminador (en este paso solo aprende el generador):
        discriminator.trainable = False

        # Entreno el sistema (en realidad solo se entrena el generador ya que
        # he congelado el discriminador):
        gloss = gan.train_on_batch(noise, yGen)

        # ** TERMINA MINI-ENTRENAMIENTO GENERADOR **


    # Store loss of most recent batch from this epoch
    dLosses.append(dloss)
    gLosses.append(gloss)
    
    if (e==1) or ((e%5)==0):
        plotGeneratedImages(e)
        if SAVE_INTERMEDIATE_DATA:
            saveModels(e)
    if (e%5)==0:
        plotLoss(e)

In [ ]:
# Plot losses from every epoch
plotLoss(e)

In [ ]:
plotGeneratedImages(e+1)

Ahora generamos un conjunto de vectores de entrada a la GAN. Cada vector de entrada tiene **randomDim** componentes:

In [ ]:
randomDim

In [ ]:
# Vamos a mostrar los resultados obtenidos para el conjunto de vectores
# de entrada en una matriz de nfilas * ncols:
nfilas = 20
ncols  = 20

# Inicializo a 0 el conjunto de vectores de entrada a la GAN:
input0 = np.zeros((nfilas*ncols, randomDim))

# Termino de calcular el conjunto de vectores de entrada.
# La idea es que en cada fila las componentes diferentes de cero
# son las mismas, y sus valores cambian de columna a columna:

nvector = 0
for i in range(nfilas):
    # Qué componentes de las randomDim se van a perturbar:
    componentes_pert = range(i,i+1)
    for j,x in enumerate(np.linspace(-4, 4, ncols)):
        input_id = i+0
        input0[nvector][componentes_pert] = x
        nvector = nvector + 1

In [ ]:
generatedImages = generator.predict(input0)
plotImages(generatedImages, nfilas, ncols, figsize=(14,14))

In [ ]:
generatedImages.shape

In [ ]:
# Para grabar las redes a fichero:

generator.save("./generator.h5")
generator.save_weights("./generator_weights.h5")
discriminator.save("./discriminator.h5")
discriminator.save_weights("./discriminator_weights.h5")
gan.save("./gan.h5")
gan.save_weights("./gan_weights.h5")